In [176]:
import urllib
import bs4
import pandas as pd
import numpy as np


In [165]:
url = "https://www.legifrance.gouv.fr/jorf/article_jo/JORFARTI000027399834"
    
from urllib import request 
request_text = request.urlopen(url).read()
print(request_text[:1000]) 
type(request_text)

b'<!DOCTYPE html><html lang="fr" class="no-js" dir="ltr"><head><title>Article  - D\xc3\xa9cret n\xc2\xb0 2013-392 du 10 mai 2013  relatif au champ d&#39;application de la taxe annuelle sur les logements vacants institu\xc3\xa9e par l&#39;article 232 du code g\xc3\xa9n\xc3\xa9ral des imp\xc3\xb4ts - L\xc3\xa9gifrance</title><meta charset="utf-8"/><meta http-equiv="X-UA-Compatible" content="IE=edge"/><meta name="viewport" content="width=device-width, initial-scale=1"/><meta name="description" content="D\xc3\xa9cret n\xc2\xb0 2013-392 du 10 mai 2013  relatif au champ d&#39;application de la taxe annuelle sur les logements vacants ... &gt; Article "/><meta name="author" content=""/><meta name="format-detection" content="telephone=no"/><meta name="_csrf" content="5065bf09-500c-402c-901e-f5e10e3732e1"/><meta name="_csrf_header" content="X-CSRF-TOKEN"/><meta name="robots" content="index, follow"><link rel="Shortcut icon" type="image/x-icon" href="/resources/images/favicon.ico"/><link rel="ico

bytes

In [167]:
page = bs4.BeautifulSoup(request_text, "lxml")

#La page ne contient qu'un seul tableau 
tableau = page.find("table")
cols = tableau.find_all('tr')

dico_zone_tlv = dict()
for col in cols:
    rows = col.find_all('td')
    rows = [ele.text.strip() for ele in rows]
    if len(rows) > 0 : #pour supprimer la première liste vide
        dico_zone_tlv[rows[2]] = rows[1] 
data_tlv = pd.DataFrame.from_dict(dico_zone_tlv,orient='index') 

print(len(data_tlv))

1148


In [168]:
data_tlv["index"] = [ i for i in range(len(data_tlv))]
data_tlv["libelle"] = data_tlv.index
data_tlv.index = data_tlv["index"]
data_tlv["tlv"] = 1
data_tlv

,0,index,libelle,tlv
index,,,,
0,2A,0,Ajaccio,1
1,74,1,Annecy,1
2,74,2,Annecy-le-Vieux,1
3,74,3,Argonay,1
4,74,4,Chavanod,1
...,...,...,...,...
1143,31,1143,L'Union,1
1144,31,1144,Vieille-Toulouse,1
1145,31,1145,Vigoulet-Auzil,1


In [169]:
from io import BytesIO
from zipfile import ZipFile
from urllib.request import urlopen

def csv_zip_to_df (url_file,file_name):
  url = urlopen(url_file)
  zipfile = ZipFile(BytesIO(url.read()))
  dataframe = pd.read_csv(zipfile.open(file_name), header = 0,sep=",")
  return dataframe

url_file = "https://www.insee.fr/fr/statistiques/fichier/4316069/cog_ensemble_2020_csv.zip"
file_name = "communes2020.csv"
com= csv_zip_to_df(url_file, file_name)




In [170]:
filter = ["01","03", "07", "15","26","38","42","43","63","69","73","74" ]
com_auv_rhones_alpes = com.query('dep == @filter')


com_auv_rhones_alpes.rename(columns={'com': 'libcom'}, inplace=True)
com_auv_rhones_alpes = com_auv_rhones_alpes[['libcom', 'libelle']]



/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [186]:
df = pd.merge(data_tlv, com_auv_rhones_alpes, on = "libelle", how = "outer")
df = df.dropna(subset=['libcom'])
df = df[["libelle", "tlv", "libcom"]]
df['tlv'] = df['tlv'].replace({np.nan : '0'})



               libelle tlv libcom
1               Annecy   1  74010
3              Argonay   1  74019
4             Chavanod   1  74067
5            Chevaline   1  74072
7             Doussard   1  74104
...                ...  ..    ...
4936   Viuz-la-Chiésaz   0  74310
4937    Viuz-en-Sallaz   0  74311
4938  Vovray-en-Bornes   0  74313
4939           Vulbens   0  74314
4940            Yvoire   0  74315

[4039 rows x 3 columns]


In [197]:
from google.colab import drive
drive.mount('drive')
df.to_csv('data_tlv.csv')
!cp data.csv "drive/My Drive/"

Mounted at drive
